Сделано на основе ноутбука от unsloth

https://unsloth.ai/blog/r1-reasoning

Установим все необходимые библиотеки  
unsloth - Для оптимизации тренировки  
vllm - для инференса модели  
tensorboard - для логирования и визуализации  
trl - библиотека для тренировки LLM 

In [1]:
!pip install unsloth vllm tensorboard
!pip install --upgrade pillow

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/user/anaconda3/envs/re-rl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


2025-02-23 12:57:35,178	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Мы хотим попробовать потренировать GRPO дома на 3090/4090 с 24ГБ видеопамяти.
Будем тренировать не всю модель, а LoRA адаптер. В таком режиме веса модели замораживаются, а тренируются дополнительные матрицы, которые затем будут добавлены в целевые веса модели.

С последними обновлениями unsloth для GRPO стало возможным использовать модели прямо с очень большим контекстом. В 3090 влезала 3B модель с 15000 контекстом.

Варьируйте параметры max_seq_len, gpu_memory_utilization если параметры установленные по-умолчанию в память не влезают.
А вообще - варьируйте все и ресечьте)

1.5B моделька с общим контекстом 456 будет трениться на всем сете GSM8K-ru примерно ~ часов.
+ если включать промежуточный евал на тестсете один прогон занимает минут 40-50.

In [ ]:
from unsloth import is_bfloat16_supported
import torch

max_seq_length = 456 # параметр задает длину контекста модели. Чем больше тем больше памяти будет требоваться и медленне тренироваться
lora_rank = 64 # LoRA ранг 64 - довольно большой, у нас получится ~120 миллионов тренируемых параметров.

# model_name = "Qwen/Qwen2.5-7B-Instruct" # Большая 7B модель
# model_name = "Qwen/Qwen2.5-3B-Instruct" # 3B модель
model_name = "Qwen/Qwen2.5-1.5B-Instruct" # 1.5B модель
# model_name = "Qwen/Qwen2.5-0.5B-Instruct" # 0.5B модель, самая слабая, но быстрее всего учится

# один из важнейших параметров далее - gpu_memory_utilization.
# расчеты из того что у нас доступно 24ГБ видеопамяти. Если меньше или больше - варьируйте значение.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # Загружаем модель в 4-бит режиме
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # сколько памяти будет занимать модель на видеокарте, можно варьировать
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # список модулей к которым применяется LoRA
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

INFO 02-23 12:57:35 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.677 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /media/user/My Passport2/hfmodels/Qwen2.5-1.5B-Instruct with actual GPU utilization = 47.0%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.68 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 456. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 8.12 GB. Also swap space = 5 GB.
INFO 02-23 12:57:40 config.py:549] This model supports multiple tasks: {'embed', 'classify', 'reward', 'g

[W223 12:57:40.657924348 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.31it/s]



INFO 02-23 12:57:41 model_runner.py:1115] Loading model weights took 2.8861 GB
INFO 02-23 12:57:41 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-23 12:57:42 worker.py:267] Memory profiling takes 0.72 seconds
INFO 02-23 12:57:42 worker.py:267] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.47) = 11.13GiB
INFO 02-23 12:57:42 worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.22GiB; the rest of the memory reserved for KV Cache is 6.96GiB.
INFO 02-23 12:57:42 executor_base.py:111] # cuda blocks: 16293, # CPU blocks: 11702
INFO 02-23 12:57:42 executor_base.py:116] Maximum concurrency for 456 tokens per request: 571.68x
INFO 02-23 12:57:43 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error 

Capturing CUDA graph shapes: 100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:12<00:00,  2.43it/s]

INFO 02-23 12:57:56 model_runner.py:1562] Graph capturing finished in 13 secs, took 1.69 GiB
INFO 02-23 12:57:56 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 14.93 seconds



Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
import re
from datasets import load_dataset, Dataset
from re_rl.tasks.math_task import MathTask  # Импорт нашего класса MathTask из библиотеки re‑rl

# Системный промт для модели который определяет формат ответа

SYSTEM_PROMPT = """
Отвечай в формате:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""


# Функция генерации датасета из задач нашей библиотеки re‑rl
def get_math_tasks_dataset(num_tasks: int = 1000) -> Dataset:
    tasks = []
    for _ in range(num_tasks):
        # Генерируем случайную задачу с помощью нашего класса
        task = MathTask.generate_random_task()
        result = task.get_result()
        # Структурируем данные так, как требуется для GRPO:
        tasks.append({
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': result["prompt"]}
            ],
            'answer': result["final_answer"],
            'solution_steps': result["solution_steps"],
            'problem': result["problem"]
        })
    return Dataset.from_list(tasks)

# Заменяем датасеты GSM8K на датасеты с математическими задачами
train_dataset = get_math_tasks_dataset(1000)
eval_dataset = get_math_tasks_dataset(200)

def extract_xml_answer(text: str) -> str:
    """Вытаскивает ответ из тегов <answer>ответ</answer>"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# Список функций наград. 
# Данные функции проверяют ответы модели на соблюдение определенный условий и
# числовое вознаграждение.
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
        Данная функция проверяет есть ли правильное решение на поставленную задачу.
        Извлекает ответ из каждого варианта.
        Если извлечённый ответ совпадает с правильным, возвращается награда 2, иначе - 0.
    """
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    """
        Для каждого извлечённого ответа проверяет, состоит ли он исключительно из цифр, используя метод isdigit().
        Если условие выполнено (то есть ответ — число), возвращается награда 0.5, иначе — 0.0
    """
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """
        Функция награды которая проверяет, что модель соблюдает указанный формат.
        Явно задаёт начало (^) и конец ($) строки. То есть вся строка должна полностью соответствовать шаблону.
    """
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """
        Еще одная функция награды которая проверяет, что модель соблюдает указанный формат.
        Не требует полного соответствия всей строки.
        Позволяет между тегами иметь произвольное количество пробельных символов.
        Не накладывает строгих требований к разбиению на строки.
    """
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [5]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True,
    vllm_gpu_memory_utilization = 0.2,
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1, # п
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1, # сколько задач в одном батче. Болье - лучше, но может не хватить памяти.
    gradient_accumulation_steps = 4, # 
    num_generations = 8, # количество ответов которые генерируются на одну задачу.
    per_device_eval_batch_size = 8,
    max_prompt_length = 256, # длина промта с задачей на входе, в токенах
    max_completion_length = 200, # максимальная длина ответа модели
    num_train_epochs = 1, # пройдет весь набор данных 1 раз
    # max_steps = 250,
    save_steps = 100, # через каждые столько шагов обучения будет сохраняться checkpoint с весами LoRA адаптера
                      # после чего можно будет продолжить обучение выставив resume_from_checkpoint = True
    max_grad_norm = 0.1,
    report_to = "tensorboard", # пишем логи в тензорборд
    output_dir = "outputs", # директория в которую пишем логи и сохраняем чекпоинты из которых затем можем продолжать обучение
    eval_strategy = "steps", # стратегия запуска тестирования на тестовой части gsm8k
    eval_steps = 300 # количество шагов трена после которого запускается прогон по тестовой выборке
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [6]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)
# resume_from_checkpoint = False значит начать обучение с начала
# выставьте resume_from_checkpoint = True чтобы продолжить обучение если сохранился хотя бы один checkpoint в директории outputs
trainer.train(resume_from_checkpoint = True)

ValueError: No valid checkpoint found in output directory (outputs)

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Сколько р в слове стравберри?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

In [ ]:
model.save_lora("grpo_saved_lora")

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Сколько р в слове стравберри?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output